In [1]:
import itertools
from collections import defaultdict

In [2]:
input_file = "23_input.txt"

with open(input_file) as f:
    initial_map = [line.rstrip() for line in f]

In [3]:
class Crater:
    def __init__(self, initial):
        self.elves_coords = defaultdict(set)
        for x, row in enumerate(initial):
            for y, c in enumerate(row):
                if c == "#":
                    self.elves_coords[x].add(y)
                    
    def neighbours(self, x, y):
        return itertools.product((x-1, x, x+1), (y-1, y, y+1))
    
    def is_empty(self, cells):
        for x, y in cells:
            if x in self.elves_coords and y in self.elves_coords[x]:
                return False
        return True
    
    
    def propose(self, x, y, nstep):
        sides = [[(x - 1, y + k) for k in range(-1, 2)]]      # north
        sides.append([(x + 1, y + k) for k in range(-1, 2)])  # south
        sides.append([(x + k, y - 1) for k in range(-1, 2)])  # west
        sides.append([(x + k, y + 1) for k in range(-1, 2)])  # east
        sides_empty = [self.is_empty(side) for side in sides]
        if all(sides_empty):
            return None
        for j in range(nstep, nstep + 4):
            if sides_empty[j % 4]:
                return sides[j % 4][1]
        return None
    
    def step(self, k):
        # propose
        proposed = dict()
        for x, row in self.elves_coords.items():
            for y in row:
                prop = self.propose(x, y, k)
                if prop:
                    if prop in proposed:
                        proposed[prop] = 0
                    else:
                        proposed[prop] = (x, y)
        # clean
        proposed = {k: v for k, v in proposed.items() if v}
        if not(proposed):
            return False
        # move
        for target in proposed:
            x, y = proposed[target]
            nx, ny = target
            self.elves_coords[x].remove(y)
            self.elves_coords[nx].add(ny)
        return True
    
    def evolve(self, n_steps):
        for k in range(n_steps):
            self.step(k)
    
    def scatter(self):
        k = 0
        has_moved = True
        while has_moved:
            has_moved = self.step(k)
            k += 1
        return k
    
    @property
    def xmin(self):
        return min(self.elves_coords)
    
    @property
    def xmax(self):
        return max(self.elves_coords)
    
    @property
    def ymin(self):
        return min(min(row) for x, row in self.elves_coords.items() if row)
    
    @property
    def ymax(self):
        return max(max(row) for x, row in self.elves_coords.items() if row)
    
    def lims(self):
        return (self.xmin, self.xmax), (self.ymin, self.ymax)
    
    
    def n_elves(self):
        return sum(len(ys) for x, ys in self.elves_coords.items())
    
    
    def print(self, chars=".#", axes=True):
        width = 1 + self.ymax - self.ymin
        if axes:
            print("  " + "".join(str(y) for y in range(self.ymin, self.ymax + 1)))
            print(" +" + '-' * width)
        for x in range(self.xmin, self.xmax + 1):
            line = [chars[0]] * width
            for y in self.elves_coords[x]:
                line[y - self.ymin] = chars[1]
            if axes:
                line = [str(x), "|"] + line
            print(''.join(line))
    
    def count_empty(self):
        area = (self.ymax - self.ymin + 1) * (self.xmax - self.xmin + 1)
        return area - self.n_elves()

In [4]:
crater = Crater(initial_map)

crater.evolve(10)
crater.count_empty()

4181

In [5]:
%%time

crater = Crater(initial_map)
crater.scatter()

CPU times: user 7.52 s, sys: 12.6 ms, total: 7.54 s
Wall time: 7.54 s


973